In [3]:
# License: Apache 2.0. See LICENSE file in root directory.
# Copyright(c) 2015-2017 Intel Corporation. All Rights Reserved.

"""
OpenCV and Numpy Point cloud Software Renderer
This sample is mostly for demonstration and educational purposes.
It really doesn't offer the quality or performance that can be
achieved with hardware acceleration.
Usage:
------
Mouse: 
    Drag with left button to rotate around pivot (thick small axes), 
    with right button to translate and the wheel to zoom.
Keyboard: 
    [p]     Pause
    [r]     Reset View
    [d]     Cycle through decimation values
    [z]     Toggle point scaling
    [c]     Toggle color source
    [s]     Save PNG (./out.png)
    [e]     Export points to ply (./out.ply)
    [q\ESC] Quit
"""

import math
import time
import cv2
import numpy as np
import pyrealsense2 as rs
import os


class AppState:

    def __init__(self, *args, **kwargs):
        self.WIN_NAME = 'RealSense'
        self.pitch, self.yaw = math.radians(-10), math.radians(-15)
        self.translation = np.array([0, 0, -1], dtype=np.float32)
        self.distance = 2
        self.prev_mouse = 0, 0
        self.mouse_btns = [False, False, False]
        self.paused = False
        self.decimate = 1
        self.scale = True
        self.color = True

    def reset(self):
        self.pitch, self.yaw, self.distance = 0, 0, 2
        self.translation[:] = 0, 0, -1

    @property
    def rotation(self):
        Rx, _ = cv2.Rodrigues((self.pitch, 0, 0))
        Ry, _ = cv2.Rodrigues((0, self.yaw, 0))
        return np.dot(Ry, Rx).astype(np.float32)

    @property
    def pivot(self):
        return self.translation + np.array((0, 0, self.distance), dtype=np.float32)


state = AppState()

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get stream profile and camera intrinsics
profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height

# Processing blocks
pc = rs.pointcloud()
decimate = rs.decimation_filter()
decimate.set_option(rs.option.filter_magnitude, 2 ** state.decimate)
colorizer = rs.colorizer()


def mouse_cb(event, x, y, flags, param):

    if event == cv2.EVENT_LBUTTONDOWN:
        state.mouse_btns[0] = True

    if event == cv2.EVENT_LBUTTONUP:
        state.mouse_btns[0] = False

    if event == cv2.EVENT_RBUTTONDOWN:
        state.mouse_btns[1] = True

    if event == cv2.EVENT_RBUTTONUP:
        state.mouse_btns[1] = False

    if event == cv2.EVENT_MBUTTONDOWN:
        state.mouse_btns[2] = True

    if event == cv2.EVENT_MBUTTONUP:
        state.mouse_btns[2] = False

    if event == cv2.EVENT_MOUSEMOVE:

        h, w = out.shape[:2]
        dx, dy = x - state.prev_mouse[0], y - state.prev_mouse[1]

        if state.mouse_btns[0]:
            state.yaw += float(dx) / w * 2
            state.pitch -= float(dy) / h * 2

        elif state.mouse_btns[1]:
            dp = np.array((dx / w, dy / h, 0), dtype=np.float32)
            state.translation -= np.dot(state.rotation, dp)

        elif state.mouse_btns[2]:
            dz = math.sqrt(dx**2 + dy**2) * math.copysign(0.01, -dy)
            state.translation[2] += dz
            state.distance -= dz

    if event == cv2.EVENT_MOUSEWHEEL:
        dz = math.copysign(0.1, flags)
        state.translation[2] += dz
        state.distance -= dz

    state.prev_mouse = (x, y)


cv2.namedWindow(state.WIN_NAME, cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow(state.WIN_NAME, w, h)
cv2.setMouseCallback(state.WIN_NAME, mouse_cb)


def project(v):
    """project 3d vector array to 2d"""
    h, w = out.shape[:2]
    view_aspect = float(h)/w

    # ignore divide by zero for invalid depth
    with np.errstate(divide='ignore', invalid='ignore'):
        proj = v[:, :-1] / v[:, -1, np.newaxis] * \
            (w*view_aspect, h) + (w/2.0, h/2.0)

    # near clipping
    znear = 0.03
    proj[v[:, 2] < znear] = np.nan
    return proj


def view(v):
    """apply view transformation on vector array"""
    return np.dot(v - state.pivot, state.rotation) + state.pivot - state.translation


def line3d(out, pt1, pt2, color=(0x80, 0x80, 0x80), thickness=1):
    """draw a 3d line from pt1 to pt2"""
    p0 = project(pt1.reshape(-1, 3))[0]
    p1 = project(pt2.reshape(-1, 3))[0]
    if np.isnan(p0).any() or np.isnan(p1).any():
        return
    p0 = tuple(p0.astype(int))
    p1 = tuple(p1.astype(int))
    rect = (0, 0, out.shape[1], out.shape[0])
    inside, p0, p1 = cv2.clipLine(rect, p0, p1)
    if inside:
        cv2.line(out, p0, p1, color, thickness, cv2.LINE_AA)


def grid(out, pos, rotation=np.eye(3), size=1, n=10, color=(0x80, 0x80, 0x80)):
    """draw a grid on xz plane"""
    pos = np.array(pos)
    s = size / float(n)
    s2 = 0.5 * size
    for i in range(0, n+1):
        x = -s2 + i*s
        line3d(out, view(pos + np.dot((x, 0, -s2), rotation)),
               view(pos + np.dot((x, 0, s2), rotation)), color)
    for i in range(0, n+1):
        z = -s2 + i*s
        line3d(out, view(pos + np.dot((-s2, 0, z), rotation)),
               view(pos + np.dot((s2, 0, z), rotation)), color)


def axes(out, pos, rotation=np.eye(3), size=0.075, thickness=2):
    """draw 3d axes"""
    line3d(out, pos, pos +
           np.dot((0, 0, size), rotation), (0xff, 0, 0), thickness)
    line3d(out, pos, pos +
           np.dot((0, size, 0), rotation), (0, 0xff, 0), thickness)
    line3d(out, pos, pos +
           np.dot((size, 0, 0), rotation), (0, 0, 0xff), thickness)


def frustum(out, intrinsics, color=(0x40, 0x40, 0x40)):
    """draw camera's frustum"""
    orig = view([0, 0, 0])
    w, h = intrinsics.width, intrinsics.height

    for d in range(1, 6, 2):
        def get_point(x, y):
            p = rs.rs2_deproject_pixel_to_point(intrinsics, [x, y], d)
            line3d(out, orig, view(p), color)
            return p

        top_left = get_point(0, 0)
        top_right = get_point(w, 0)
        bottom_right = get_point(w, h)
        bottom_left = get_point(0, h)

        line3d(out, view(top_left), view(top_right), color)
        line3d(out, view(top_right), view(bottom_right), color)
        line3d(out, view(bottom_right), view(bottom_left), color)
        line3d(out, view(bottom_left), view(top_left), color)


def pointcloud(out, verts, texcoords, color, painter=True):
    """draw point cloud with optional painter's algorithm"""
    if painter:
        # Painter's algo, sort points from back to front

        # get reverse sorted indices by z (in view-space)
        # https://gist.github.com/stevenvo/e3dad127598842459b68
        v = view(verts)
        s = v[:, 2].argsort()[::-1]
        proj = project(v[s])
    else:
        proj = project(view(verts))

    if state.scale:
        proj *= 0.5**state.decimate

    h, w = out.shape[:2]

    # proj now contains 2d image coordinates
    j, i = proj.astype(np.uint32).T

    # create a mask to ignore out-of-bound indices
    im = (i >= 0) & (i < h)
    jm = (j >= 0) & (j < w)
    m = im & jm

    cw, ch = color.shape[:2][::-1]
    if painter:
        # sort texcoord with same indices as above
        # texcoords are [0..1] and relative to top-left pixel corner,
        # multiply by size and add 0.5 to center
        v, u = (texcoords[s] * (cw, ch) + 0.5).astype(np.uint32).T
    else:
        v, u = (texcoords * (cw, ch) + 0.5).astype(np.uint32).T
    # clip texcoords to image
    np.clip(u, 0, ch-1, out=u)
    np.clip(v, 0, cw-1, out=v)

    # perform uv-mapping
    out[i[m], j[m]] = color[u[m], v[m]]



out = np.empty((h, w, 3), dtype=np.uint8)

def get_verts(name):
    npsavePath='C://Users//user//Desktop//openPose//pointcloud/'+name
    pc_token = np.load(npsavePath)
    return pc_token

def zero_filter(x):
    if (x[2] > 1.5) and (x[2] < 3):
        return [x[0],x[1],x[2]]
    else:
        return [0,0,0]
    
def binary(x):
    if (x == 0):
        return 0
    else:
        return 0.5
   

In [4]:
framecount =0

# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2

# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = 1 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)


Depth Scale is:  0.0010000000474974513


In [9]:
i=0
sums=np.zeros((76800, 3))
u_sum=np.zeros((76800, 3))
framecount =0

while True:
    # Grab camera data
    if not state.paused:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()

        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_frame = decimate.process(depth_frame)

        # Grab new intrinsics (may be changed by decimation)
        depth_intrinsics = rs.video_stream_profile(
            depth_frame.profile).get_intrinsics()
        w, h = depth_intrinsics.width, depth_intrinsics.height

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        depth_colormap = np.asanyarray(
            colorizer.colorize(depth_frame).get_data())

        if state.color:
            mapped_frame, color_source = color_frame, color_image
        else:
            mapped_frame, color_source = depth_frame, depth_colormap
        
        points = pc.calculate(depth_frame)
        pc.map_to(mapped_frame)
        
        # Pointcloud data to arrays
        v, t = points.get_vertices(), points.get_texture_coordinates()
        verts = np.asanyarray(v).view(np.float32).reshape(-1, 3)  # xyz
        npsavePath='C://Users//user//Desktop//openPose//outputs//pointcloud/'+str(framecount)
        np.save(npsavePath,verts)
        
        texcoords = np.asanyarray(t).view(np.float32).reshape(-1, 2)  # uv
        tcsavePath='C://Users//user//Desktop//openPose//outputs//texcoords/'+str(framecount)
        #np.save(tcsavePath,texcoords)
        
        plysavePath='C://Users//user//Desktop//openPose//outputs//ply/'+str(framecount)+'.ply'
        #points.export_to_ply(plysavePath, mapped_frame)
        
        ###############################################
        
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            print('Error, fail to get depth or color frame')
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        savePath_d = 'C://Users//user//Desktop//openPose//outputs//test_depth_array/' + str(framecount).zfill(5)
        savePath_c = 'C://Users//user//Desktop//openPose//outputs//test_color_array/' + str(framecount).zfill(5)
        np.save(savePath_d,depth_image)
        np.save(savePath_c,color_image)#1개의 배열을 NumPy format의 바이너리 파일로 저장하기 (Save a single array to a binary file in NumPy format)
        
        # Render images
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        images = np.hstack((color_image, depth_colormap))
        cv2.namedWindow('Align Example', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('Align Example', images)
        
        key = cv2.waitKey(1)
        print('frame : ',framecount)
        framecount = framecount+1
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break

# Stop streaming

frame :  0
frame :  1
frame :  2
frame :  3
frame :  4
frame :  5
frame :  6
frame :  7
frame :  8
frame :  9
frame :  10
frame :  11
frame :  12
frame :  13
frame :  14
frame :  15
frame :  16
frame :  17
frame :  18
frame :  19
frame :  20
frame :  21
frame :  22
frame :  23
frame :  24
frame :  25
frame :  26
frame :  27
frame :  28
frame :  29
frame :  30
frame :  31
frame :  32
frame :  33
frame :  34
frame :  35
frame :  36
frame :  37
frame :  38
frame :  39
frame :  40
frame :  41
frame :  42
frame :  43
frame :  44
frame :  45
frame :  46
frame :  47
frame :  48
frame :  49
frame :  50
frame :  51
frame :  52
frame :  53
frame :  54
frame :  55
frame :  56
frame :  57
frame :  58
frame :  59
frame :  60
frame :  61
frame :  62
frame :  63
frame :  64
frame :  65
frame :  66
frame :  67
frame :  68
frame :  69
frame :  70
frame :  71
frame :  72
frame :  73
frame :  74
frame :  75
frame :  76
frame :  77
frame :  78
frame :  79
frame :  80
frame :  81
frame :  82
frame :  83
fr

In [10]:
import os
import numpy as np
import cv2

videopath = 'C://Users//user//Desktop//openPose//outputs//test_color_array/'
frame_list = sorted(os.listdir(videopath)) #피험자 리스트
print(frame_list)

outputVideo = []

for j in frame_list :#frame별 slice
    open_frame = videopath +'//' + j
    save_load = np.load(open_frame)
    cv2.imwrite('color_img.jpg', save_load)
    print(j,save_load.shape)
    #cv2.imwrite('color_img.jpg', save_load)
    outputVideo.append(save_load)


out = cv2.VideoWriter('C://Users/user/Desktop/openPose//outputs/raw_video(10fps).avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (640,480))

for i in range(len(outputVideo)):
    out.write(outputVideo[i])
              
out.release()

['00000.npy', '00001.npy', '00002.npy', '00003.npy', '00004.npy', '00005.npy', '00006.npy', '00007.npy', '00008.npy', '00009.npy', '00010.npy', '00011.npy', '00012.npy', '00013.npy', '00014.npy', '00015.npy', '00016.npy', '00017.npy', '00018.npy', '00019.npy', '00020.npy', '00021.npy', '00022.npy', '00023.npy', '00024.npy', '00025.npy', '00026.npy', '00027.npy', '00028.npy', '00029.npy', '00030.npy', '00031.npy', '00032.npy', '00033.npy', '00034.npy', '00035.npy', '00036.npy', '00037.npy', '00038.npy', '00039.npy', '00040.npy', '00041.npy', '00042.npy', '00043.npy', '00044.npy', '00045.npy', '00046.npy', '00047.npy', '00048.npy', '00049.npy', '00050.npy', '00051.npy', '00052.npy', '00053.npy', '00054.npy', '00055.npy', '00056.npy', '00057.npy', '00058.npy', '00059.npy', '00060.npy', '00061.npy', '00062.npy', '00063.npy', '00064.npy', '00065.npy', '00066.npy', '00067.npy', '00068.npy', '00069.npy', '00070.npy', '00071.npy', '00072.npy', '00073.npy', '00074.npy', '00075.npy', '00076.npy'